# Convolutional Neural Network

In [1]:
# Image Input-->[feature detector + ReLU][Convolution]-->Convolutional layer-->[Max Pooling]-->Pooling layer-->Flattening->ANN
# In CNN, not only the weights are adjusted in backpropagation but the feature detectors are also adjusted

#CNN to be implemented in Jupyter Notebook because of the size of the dataset

### Importing the libraries

In [2]:
#we are dealing with only images so the preprocessing stage is kinda different from the classic way
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
#we have to apply transformations on all images of the training set only.
#this is to avoid overfitting
#What are these transformations--> Zooms, Rotations, Flip, Shear, etc [Coursera]. The process is called Image Augmentation

# https://keras.io/api/preprocessing/image/

train_datagen = ImageDataGenerator(rescale = 1./255,  #applies feature scalling to pixels by dividing all by 255 to give a range of 0-1
                                   shear_range = 0.2, #taken from keras api
                                   zoom_range = 0.2,  #taken from keras api
                                   horizontal_flip = True)  #taken from keras api
#Above is the object taken downloaded from keras 
#we now need to connect this to our training set images, also copied from Keras        
#NOT SURE OF THE LOCATION                           
training_set = train_datagen.flow_from_directory('dataset/training_set',#using jupyter notebook  #a path to our training set
                                                 target_size = (64, 64), #150, 150 made the training process very long
                                                 batch_size = 32, #number of images we want in each batch. 32 is default
                                                 class_mode = 'binary') #cat or dog, so we chose binary

### Preprocessing the Test set

In [ ]:
#test transformation isn't applied to test set
#only the pixels must be rescaled
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',#using jupyter notebook
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) 
#classic filter is 32 [32 in first convulational layer and 32 for the next conv layer], you can tweak it
#kernel size = feature detector being 3x3 matrix
#[64, 64, 3] this is for coloured pics. For B/W, change 3 to 1
#usng 64,64 because we changed our target_size to 64,64
#using Rectifier Activation function

### Step 2 - Pooling

In [ ]:
#applying max pooling, also in layers module
#A 2x2 matrix was applied to the Feature Map to obtain the Pooled Feature Map
#A stride of 2 was applied
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
#copied steps 1 and 2 and just deleted the Input Shape
#using Rectifier Activation function
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
#adding a fully connected neural network
#we increased the number of hiddent neurons to 128 instead 6 like in the ANN
#using Rectifier Activation function
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
#adding a Fully Connected Output layer
#Just one output because its binary classification
#activation for output for this classifier should be SIGMOID instead of SOFTMAX
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
#still using binary classification
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
#training and testing happen at the same time
#still using the fit method
#training set indicated in preprocessing step
#10 wasn't enough, 15 also wasn't converging. 
#25 was better, taking about 15-20 minutes
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)


## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
#load the single image
#image.load_img loads image as PIL
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
#converting the PIL image to a numpy array required by the predict method
test_image = image.img_to_array(test_image)
#adding an extra dimension to image which corresponds to the batch size of images  [each batch has 32 images, abatch size = 32]
test_image = np.expand_dims(test_image, axis = 0)
#result here gives either a 0 or 1 response
result = cnn.predict(test_image)
#writing our code to intepret the results.
#class_indices is from keras
training_set.class_indices
if result[0][0] == 1: #indexing depends on location of data
       #first [0] = the batch
       #second [0] = the one and only element in the batch (Prediction folder) 
       
       #HOW DID YOU KNOW THAT 1 IS FOR DOG??

  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)

In [ ]:
#download ipnyb file